In [1]:
import os
import ccxt
import configparser
import vectorbt as vbt
from datetime import datetime
import pytz
import json
import numpy as np
import talib as ta
import pandas as pd
import schedule
import time
import warnings
import requests
import decimal
warnings.filterwarnings('ignore')

#pd.set_option('display.max_rows', None)

# TODO
- แก้ fetch response erorr ให้ try ตาม count
- Takeprofit ,Trailing Stop
- TEST 15m,30m,1h ,4h  TIMEFRAME
-แก้ LOGS เป็น entry_df ,exit_df ใช้แทน

# EDIT, CREATES
- แก้ฟังชั่น get_postions , placeorderlong ,short , closeposition ,PORTLOG[CSVFILES] ,
- เพิ่มฟังชั่น read_trades_df สำหรับ history trades
- เพิ่ม close_trades สำหรับตอนเข้าออร์เดอร์โดยบอท , บันทึกลง CSV history trades ** บันทึกแต่ Orders ที่ entry by bo
- ถ้ามี Positions อยู่แล้วตอนเปิดบอท entry_df =pd.DataFrame() จะไม่รู้เวลา TS ที่จะ FETCH TRADES มา groupby หาวิธียังไม่ได้เลยไม่บันทึก orders ครับ
- 
# 

#### Global configuration

In [2]:
config = configparser.ConfigParser()
config.read('key.ini')
# --------------------------------------------------
conf_robot_name = 'ActionZone'
conf_api_key = config['key']['apikey'] 
conf_api_secret = config['key']['secretkey']
conf_symbol = 'BTC-PERP' # trade symbol
conf_timeframe = '1m' # only support timeframe: 1m, 3m, 5m, 15m, 1h
conf_max_candles = 100 # total candles to be loaded from exchange, max is 1,000
conf_riskpertrade = 0.0001 # 1%
conf_position_size_limit = 1 # max position size to allow to trade
conf_leverage = 20
#conf_max_retry = 10 # max retry to get data
# --------------------------------------------------
conf_line_token = config['key']['line_token']
conf_line_url = 'https://notify-api.line.me/api/notify'
conf_line_headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+conf_line_token}
# --------------------------------------------------
exchange = ccxt.ftx({
    'apiKey' : conf_api_key ,
    'secret' : conf_api_secret ,
    'enableRateLimit': True,
    'option' : {'defaultType' : 'future', 'adjustForTimeDifference': True}
})
exchange.headers = {
    'FTX-SUBACCOUNT': 'testAPI',
}

LOGFILE = f'{conf_robot_name}_log.txt' 
PORT_LOG = 'trades_history.csv'

response = exchange.private_post_account_leverage({'leverage': conf_leverage,}) 
print(exchange.fetchStatus())


{'status': 'ok', 'updated': None, 'eta': None, 'url': None}


In [3]:
def line_notify(msg):
    r = requests.post(conf_line_url, headers=conf_line_headers, data ={'message':str(msg)})
    r.text

def get_time():    
    now = datetime.now()
    formatted_date = now.strftime("%d/%m/%Y %H:%M:%S")
    return formatted_date

def get_wallet():
    try:
        wallet = exchange.privateGetWalletBalances()['result']
        return wallet
    except:
        return None

def get_cash():
    try:
        wallet = get_wallet()
        for t in wallet:
            if t['coin'] in  ['USD']:
                cash = float(t['availableWithoutBorrow'])
        return cash
    except:
        return None
    
def get_position(symbols): # แก้
    try:
        res = exchange.fetchPositions()
        for sym in res:
            if sym['info']['future']==symbols:
                f_pos = sym['info']
        return f_pos
    except Exception as e:
        log(str(e),LOGFILE)
        return None     
    
def get_price_digit(symbol): # นับ digit จาก fetch ticker ล่าสุด
    try:
        res =exchange.fetch_ticker(symbol)# ['info']['minProvideSize']
        price_step = float(res['info']['priceIncrement'])
        str_digit = str(price_step)
        count_digit = len(str_digit.split('.')[1])
        return count_digit
    except :
        pass
    
def get_size_digit(symbol): # นับ digit จาก fetch ticker ล่าสุด
    try:
        res =exchange.fetch_ticker(symbol)# ['info']['minProvideSize']
        size_step = float(res['info']['sizeIncrement'])
        last_price = float(res['info']['last'])
        min_provide = size_step * last_price
        str_digit = str(size_step)
        count_digit = len(str_digit.split('.')[1])

        return count_digit
    except :
        pass    
 
def get_minimum_size(symbol): ##จะไม่ได้ต้องเซ็ต fetch minimum จาก exchange กัน error
    res = exchange.fetch_ticker(symbol)['info']#['sizeIncrement']
    minimum_size = float(res['sizeIncrement'])
    last_bid = float(res['bid'])
    min_value =minimum_size * last_bid
    return float(minimum_size)

conf_min_size = get_minimum_size(conf_symbol)           

In [4]:
def reset_trade_log():
    latest_log['symbol'] = ''
    latest_log['entry_time'] = 0 
    latest_log['entry_price'] = 0.0 # average open price
    latest_log['position_side'] = '' # LONG or SHORT
    latest_log['position_amount'] = 0.0 # position size
    latest_log['stop_loss'] = 0.0 # stop loss price
    latest_log['take_profit'] = 0.0 # take profit price 
    
    
def load_trades_log(df):
    last_position = check_positions()
    if last_position == 1:
        f_pos = get_position(conf_symbol)
        latest_log['symbol'] =  f_pos['future']
        latest_log['entry_time'] = df['timestamp'].iloc[-1]
        latest_log['entry_price'] =  float(f_pos['entryPrice'])  
        latest_log['position_side'] = f_pos['side'] 
        latest_log['position_amount'] =  float(f_pos['size'])
        latest_log['stop_loss'] = float(f_pos['entryPrice']) - Cal_SLdistance(df)  
        latest_log['take_profit'] = 0.0
    
    elif last_position == -1:
        f_pos = get_position(conf_symbol)
        latest_log['symbol'] =  f_pos['future']
        latest_log['entry_time'] = df['timestamp'].iloc[-1]
        latest_log['entry_price'] =  float(f_pos['entryPrice'])  
        latest_log['position_side'] = f_pos['side'] 
        latest_log['position_amount'] =  float(f_pos['size'])
        latest_log['stop_loss'] = float(f_pos['entryPrice']) + Cal_SLdistance(df)  
        latest_log['take_profit'] = 0.0
    else: # Postion == 0
        reset_trade_log()
    return latest_log    

In [5]:
def append_log(cur_bar_str, last_position, last_price):
    if last_position ==1:
        diff = last_price - latest_log['entry_price']
        pnl = diff * latest_log['position_amount']
        
    if last_position ==-1:
        diff = latest_log['entry_price'] - last_price
        pnl = diff * latest_log['position_amount']
        
    line = '{},{},{},{},{},{},{},{}\n'.format(
        latest_log['symbol'], latest_log['entry_time'], latest_log['entry_price'], latest_log['position_side']
        , latest_log['position_amount'], cur_bar_str, last_price, pnl
    )
    f = open(LOGFILE, "a")
    f.write(line + "\n")
    f.close()

    
def writes_trades_log():
    line = 'symbol, entry_time, entry_price, position_side, position_amount, exit_time, exit_price, pnl'
    f = open(LOGFILE, 'w')
    f.write(line + "\n")
    f.close()     
    
    

In [6]:
def Cal_SLdistance(df, vol_multiply = 1.2):
    vol = ta.STDDEV(df.close,30).ewm(alpha=0.96).mean()
    sl_distance = vol * vol_multiply
    return sl_distance.iloc[-1]

def Cal_TPdistance(df, vol_multiply = 1.2):
    vol = ta.STDDEV(df.close,30).ewm(alpha=0.96).mean()
    tp_distance = vol * vol_multiply
    return tp_distance.iloc[-1]

def Cal_Size(df, conf_riskpertrade = conf_riskpertrade):
    cash =get_cash()
    if cash != None:
        riskpertrade = conf_riskpertrade * cash
        size = (riskpertrade / Cal_SLdistance(df)) 
        if size > conf_position_size_limit:
            size = conf_position_size_limit
        elif size < conf_min_size:
            size = conf_min_size    
    return round(size, get_size_digit(conf_symbol))          

In [7]:
def read_trades_df():
    try:
        df= pd.read_csv(PORT_LOG)
        print('--DataBase Exist Loading DB---')

    except:
        df= pd.DataFrame(columns=['symbol','timestamp','side','price','amount','cost'])
        df.to_csv(PORT_LOG, index=False)
        print("---DataBase Created---")
        
    return df 

def create_open_market_order(symbols:str, side:str,size:float ,params={}):  ### Custom param
    try:
            orderInfo = exchange.create_order(conf_symbol, 'market', side, size,params=params)["info"]
            orderID = orderInfo["id"]
            entry_ts =exchange.parse8601(orderInfo['createdAt'])
            print('Time sleep To Fetch Trades') # ถ้าไม่รอมีปัญหาตรง res_trades ไม่เจอ เป็น [] เปล่า
            time.sleep(3)
            res_trades = exchange.fetch_my_trades(symbols, since=int(entry_ts))
            if res_trades != []:
                entry_df_ts  = pd.DataFrame(res_trades)
                entry_df  =entry_df_ts.groupby('symbol').agg({'timestamp':'last','side':'last','price':'mean','amount':'sum','cost':'sum'})

                price = entry_df['price'][0]
            usdValue = round(price * size, 3)
            msg = f'Open-{orderID} {side} ,{symbols} Price: {price:.5f} Unit :{size} Usd {usdValue}$'
            return entry_df
    except ccxt.InvalidOrder as e:
        pass
    except ccxt.NetworkError as e:
        pass
    except ccxt.ExchangeError as e:
        pass
    except ccxt.InsufficientFunds as e:
        pass

    
def close_positions_edit(symbols=conf_symbol):    
    close_df = pd.DataFrame()

    netsize = float(get_position(symbols)['netSize'])
    print(netsize)
    if netsize > 0:
        close_df = create_open_market_order(symbols,side='sell',size=abs(netsize),params={'conditionalOrdersOnly':True}) 
        close_df = close_df.add_suffix('_exit')

        msg = '------ Close Long ------'
        line_notify(msg) 
        
    elif netsize < 0:  
        close_df  = create_open_market_order(symbols,side='buy',size=abs(netsize),params={'conditionalOrdersOnly':True})
        close_df=close_df.add_suffix('_exit')
        # 
        
        msg = '------ Close Short ------'
        line_notify(msg)
        
    else :
        print('No positions to close')
    return close_df    

In [8]:

def close_trades(entry_df,exit_df):
    
    if not entry_df.empty:
        trades_df = pd.concat([entry_df,exit_df],axis=1)
        side_= trades_df['side'][0]
        if side_ =='buy':
            trades_df['diff_price'] = trades_df['price_exit'] - trades_df['price'] 
            trades_df['pnl'] = trades_df['cost_exit'] - trades_df['cost'] 
            trades_df =trades_df.reset_index()

        else:
            trades_df['diff_price'] =  trades_df['price'] -trades_df['price_exit']
            trades_df['pnl'] =  trades_df['cost'] - trades_df['cost_exit'] 
            trades_df =trades_df.reset_index()
    else:# Returns Empty DataFrame
        trades_df =pd.DataFrame()
    return trades_df


def check_positions():
    res = exchange.private_get_positions()['result']   
    for symbol in res:
        if symbol['future'] in conf_symbol:
            netsize = float(symbol['netSize'])
            if netsize > 0:
                positions = 1 # Have Long Positions
            elif netsize < 0:
                positions = -1 # Have Short Positions
            else :
                positions = 0 # No Positions
            return positions  

In [9]:
def is_new_bar(prev_bar_time, cur_bar_time):
    if cur_bar_time > prev_bar_time:
        return True
    else:
        return False  
    
def fetch_data(symbols = conf_symbol, timeframe = conf_timeframe, limit = conf_max_candles):
    try_count = 0 
    try:        
        bars = exchange.fetch_ohlcv(symbols, timeframe, limit = limit)
        df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except :
        print('LOAD DATA ERROR ')
        pass
        return pd.DataFrame()

def strategy(df):    
    if (not df.empty) :
        
        df['ema1'] = ta.EMA(df.close, 12)
        df['ema2'] = ta.EMA(df.close, 26)
        
        df['LongEntries'] = (df['ema1'] > df['ema2']) & (df['ema1'].shift(1) < df['ema2']) & (df.close > df['ema1'])
        df['LongExit'] = df.close < df['ema2']   
        df['ShortEntries'] = (df['ema1'] < df['ema2']) & (df['ema1'].shift(1) > df['ema2']) & (df.close < df['ema1'])
        df['ShortExit'] = df.close > df['ema2'] 
        return df
    
    else:
        print('Cant Fetch Data' )
        return df

In [10]:
def trading():
    global prev_bar
    global entry_df
    global exit_df
    global all_trades
    now_ts = int(exchange.milliseconds()) #
    now_dt=  datetime.fromtimestamp(int(now_ts/1000)) 

    if prev_bar != 0:
        prev_bar_str = datetime.utcfromtimestamp(int(prev_bar/1000))
    else:
        prev_bar_str= []
        
    df_raw = fetch_data()
    df = strategy(df_raw) # add fetchingdata to strategy
    print(f'RUN BOT  @ {now_dt} ')
    if  not df.empty:
        LongEntries = df['LongEntries'].iloc[-1] #[last_row_index]
        LongExit = df['LongExit'].iloc[-1]   
        ShortEntries = df['ShortEntries'].iloc[-1]
        ShortExit = df['ShortExit'].iloc[-1]
        size = Cal_Size(df)
        
        print('Long : ',LongEntries,LongExit )    
        print('SHORT: ',ShortEntries,ShortExit )
        
        cur_bar_str =   df.timestamp.iloc[-1]
        cur_bar_time = int(cur_bar_str.timestamp()*1000) #

        new_bar_cond = is_new_bar(prev_bar,cur_bar_time)
        
        last_position = check_positions()
        last_price = df['close'].iloc[-1]
        
        
        print(f" Latest Position : {latest_log}")     

        print('----------------------------------------')
        print(f"Fetching new bars for  {prev_bar_str} : {cur_bar_str} :")

        print(f"EMA12 : {df['ema1'].iloc[-1]}, EMA26 : {df['ema2'].iloc[-1]}  Last Price {last_price}")
        print(f"Position : {last_position}")
        
        if (prev_bar != cur_bar_time) :
            print('NEW BAR')
            if last_position == 0: 

                if LongEntries == True:    
                    # place_longposition(df)   
                    entry_df = create_open_market_order(conf_symbol,'buy',size=size)

                    load_trades_log(df) 
                    print("------ Open Long ------")

                elif ShortEntries == True: 
                    # place_shortposition(df)  
                    entry_df = create_open_market_order(conf_symbol,'sell',size=size)

                    load_trades_log(df)                                                                            
                    print("------ Open Short ------")

                else :
                    print('No Positions and Signals')

                    
##### CLOSED POSITION ################### 
            elif last_position == 1: 
                if LongExit == True :
                    # append_log(cur_bar_str, last_position, last_price)
                    # close_positions()
                    exit_df =close_positions_edit(conf_symbol)
                    trades_df =close_trades(entry_df,exit_df)
                    if not trades_df.empty:
                            all_trades = all_trades.append(trades_df).set_index('symbol')
                            all_trades.to_csv(PORT_LOG)
                            print('RECORD TRADES')
                            
                    load_trades_log(df)
                    print("------ Exit Long ------")

                elif  (latest_log['take_profit'] != 0.0) and (last_price >= latest_log['take_profit']):
                    # append_log(cur_bar_str, last_position, last_price)
                    exit_df =close_positions_edit(conf_symbol)
                    
                    trades_df =close_trades(entry_df,exit_df)
                    print(trades_df)

                    if not trades_df.empty:
                            all_trades = all_trades.append(trades_df).set_index('symbol')
                            all_trades.to_csv(PORT_LOG)
                            print('RECORD TRADES')
                        
                    load_trades_log(df)
                    print('TAKE PROFIT Long')

                elif  (latest_log['stop_loss'] != 0.0) and (last_price <= latest_log['stop_loss']):
                    # append_log(cur_bar_str, last_position, last_price)
                    exit_df =close_positions_edit(conf_symbol)
                    trades_df =close_trades(entry_df,exit_df)
                    if not trades_df.empty:
                            all_trades = all_trades.append(trades_df).set_index('symbol')
                            all_trades.to_csv(PORT_LOG)
                            print('RECORD TRADES')
                        
                    load_trades_log(df)
                    print('STOPLOSS Long')

                elif ShortEntries == True: # Reverse Signal
                    # append_log(cur_bar_str, last_position, last_price)
                    exit_df =close_positions_edit(conf_symbol)
                    # เพิ่ม
                    trades_df =close_trades(entry_df,exit_df)
                    if not trades_df.empty:
                            all_trades = all_trades.append(trades_df).set_index('symbol')
                            all_trades.to_csv(PORT_LOG)
                            print('RECORD TRADES')
                            
                    load_trades_log(df)
                    # place_shortposition(df) 
                    entry_df = create_open_market_order(conf_symbol,'sell',size=size)
                    load_trades_log(df)
                    print("------ Open Short ------")
                else:
                    print('CHECK PNL')
                    print('Have Long Positions, No signal')
                    print(latest_log)


            elif last_position== -1: # Have Short Positions 

                if ShortExit == True :
                    # append_log(cur_bar_str, last_position, last_price)
                    # close_positions()
                    exit_df =close_positions_edit(conf_symbol)
                    # เพิ่ม
                    trades_df =close_trades(entry_df,exit_df)
                    print(trades_df)
                    if not trades_df.empty:
                            all_trades = all_trades.append(trades_df).set_index('symbol')
                            all_trades.to_csv(PORT_LOG)
                            print('RECORD TRADES')
                                           
                    load_trades_log(df)                
                    print("------ Exit Short ------")

                elif  (latest_log['take_profit'] != 0.0) and (last_price <= latest_log['take_profit']):
                    # append_log(cur_bar_str, last_position, last_price)
                    # close_positions()
                    exit_df =close_positions_edit(conf_symbol)
                    # เพิ่ม
                    trades_df =close_trades(entry_df,exit_df)
                    if not trades_df.empty:
                            all_trades = all_trades.append(trades_df).set_index('symbol')
                            all_trades.to_csv(PORT_LOG)
                            print('RECORD TRADES')
                        
                    load_trades_log(df)                
                    print('TAKE PROFIT Short')
                    
                elif  (latest_log['stop_loss'] != 0.0) and (last_price >= latest_log['stop_loss']):
                    # append_log(cur_bar_str, last_position, last_price)
                    # close_positions()
                    exit_df =close_positions_edit(conf_symbol)
                    # เพิ่ม
                    trades_df =close_trades(entry_df,exit_df)
                    if not trades_df.empty:
                            all_trades = all_trades.append(trades_df).set_index('symbol')
                            all_trades.to_csv(PORT_LOG)
                            print('RECORD TRADES')
                        
                    load_trades_log(df) 
                    print('STOPLOSS Short')                                       

                elif LongEntries == True: # Reverse Signal
                    # append_log(cur_bar_str, last_position, last_price)
                    # close_positions()
                    exit_df =close_positions_edit(conf_symbol)
                    # เพิ่ม
                    trades_df =close_trades(entry_df,exit_df)
                    if not trades_df.empty:
                            all_trades = all_trades.append(trades_df).set_index('symbol')
                            all_trades.to_csv(PORT_LOG)
                            print('RECORD TRADES')
                    load_trades_log(df)
                    # place_longposition(df)
                    entry_df = create_open_market_order(conf_symbol,'sell',size=size)
                    load_trades_log(df)
                    print("------ Open Long ------")

                else:
                        print('CHECK PNL')
                        print('Have Short Positions, No signal')
                        print(latest_log)

        else:
            print('SAMEBAR')
            
        prev_bar = cur_bar_time
        print('-'*50)
        
    else:
        print('Can Not get DataFrame')
        pass

In [11]:
writes_trades_log()

In [ ]:
from IPython.display import clear_output
latest_log = {}
latest_log['symbol'] = ''
latest_log['entry_time'] = 0 
latest_log['entry_price'] = 0.0 # average open price
latest_log['position_side'] = '' # LONG or SHORT
latest_log['position_amount'] = 0.0 # position size
latest_log['stop_loss'] = 0.0 # stop loss price
latest_log['take_profit'] = 0.0 # take profit price
count = 0
prev_bar = 0
writes_trades_log()
all_trades = read_trades_df() # เพิ่ม
entry_df = pd.DataFrame()
exit_df = pd.DataFrame()
print(f'{conf_symbol} BOT  Time Frame{ conf_timeframe} RPT {conf_riskpertrade*100:.2f} MAX LEVERAGE{conf_leverage}')
while True:
    trading()
    count += 1
    time.sleep(30)
    if count == 1000:
        count = 0
        clear_output(wait=True) 

RUN BOT  @ 2022-03-22 18:47:36 
Long :  False False
SHORT:  False True
 Latest Position : {'symbol': 'BTC-PERP', 'entry_time': Timestamp('2022-03-22 11:44:00'), 'entry_price': 42972.0, 'position_side': 'buy', 'position_amount': 0.0001, 'stop_loss': 42913.85735546023, 'take_profit': 0.0}
----------------------------------------
Fetching new bars for  2022-03-22 11:46:00 : 2022-03-22 11:46:00 :
EMA12 : 42927.11458415957, EMA26 : 42918.1090743944  Last Price 42957.0
Position : 1
SAMEBAR
--------------------------------------------------
RUN BOT  @ 2022-03-22 18:48:07 
Long :  False False
SHORT:  False True
 Latest Position : {'symbol': 'BTC-PERP', 'entry_time': Timestamp('2022-03-22 11:44:00'), 'entry_price': 42972.0, 'position_side': 'buy', 'position_amount': 0.0001, 'stop_loss': 42913.85735546023, 'take_profit': 0.0}
----------------------------------------
Fetching new bars for  2022-03-22 11:46:00 : 2022-03-22 11:47:00 :
EMA12 : 42929.096956555644, EMA26 : 42919.722347901145  Last Pri